In [2]:
#!pip install crewai

In [24]:
import boto3
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from typing import Any
from pydantic import Field
import logging
import json

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from cred import *

In [25]:
litellm._turn_on_debug()

In [12]:
# Flask app initialization
app = Flask(__name__)

# Custom Tool for Bedrock Knowledge Base Search
class BedrockKnowledgeBaseTool(BaseTool):
    name: str = "Bedrock Knowledge Base Search"
    description: str = "Searches an AWS Bedrock Knowledge Base for relevant information based on a query."
    
    # Define Pydantic fields for custom attributes
    knowledge_base_id: str = Field(..., description="The ID of the Bedrock Knowledge Base")
    region: str = Field(default="us-east-1", description="AWS region for the Bedrock client")

    def __init__(self, knowledge_base_id: str, region: str = "eu-north-1"):
        # Pass Pydantic fields to the parent class
        super().__init__(knowledge_base_id=knowledge_base_id, region=region)
        # Initialize boto3 client as a private attribute to avoid Pydantic validation
        self._bedrock_client = boto3.client("bedrock-agent-runtime", region_name=region)


    def _run(self, query: str) -> str:
        
        #return "No relevant information found in the knowledge base."
        try:
            response = self._bedrock_client.retrieve(
                knowledgeBaseId=self.knowledge_base_id,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            )
            results = response.get("retrievalResults", [])
            if not results:
                return "No relevant information found in the knowledge base."
            answer = "\n".join([result["content"]["text"] for result in results])
            return answer
        except Exception as e:
            return f"Error querying Bedrock Knowledge Base: {str(e)}"

In [26]:
class BedrockKnowledgeBaseTool(BaseTool):
    name: str = "Bedrock Knowledge Base Search"
    description: str = "Searches an AWS Bedrock Knowledge Base for relevant information based on a query."
    knowledge_base_id: str = Field(..., description="The ID of the Bedrock Knowledge Base")
    region: str = Field(default="eu-north-1", description="AWS region for the Bedrock client")

    def __init__(self, knowledge_base_id: str, region: str = "eu-north-1"):
        super().__init__(knowledge_base_id=knowledge_base_id, region=region)
        try:
            session = boto3.Session()
            self._bedrock_client = session.client("bedrock-agent-runtime", region_name=region)
            logger.info(f"Initialized Bedrock client for region {region}")
        except Exception as e:
            logger.error(f"Failed to initialize Bedrock client: {str(e)}")
            raise

    def _run(self, query_input: Any) -> str:
        try:
            # Handle different input types
            if isinstance(query_input, str):
                try:
                    # Check if input is a JSON string
                    query_dict = json.loads(query_input)
                    query = query_dict.get('query', '')
                    logger.warning(f"Parsed JSON string input, extracted query: {query}")
                except json.JSONDecodeError:
                    # Treat as direct string query
                    query = query_input
            elif isinstance(query_input, dict):
                query = query_input.get('query', '') or query_input.get('description', '')
                logger.warning(f"Received dict input, extracted query: {query}")
            else:
                raise ValueError(f"Invalid query type: must be string or dict, got {type(query_input)}")

            # Validate query
            if not isinstance(query, str) or not query.strip():
                raise ValueError(f"Invalid query: must be a non-empty string, got {query}")

            response = self._bedrock_client.retrieve(
                knowledgeBaseId=self.knowledge_base_id,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            )
            results = response.get("retrievalResults", [])
            if not results:
                return "No relevant information found in the knowledge base."
            answer = "\n".join([result["content"]["text"] for result in results])
            return answer
        except Exception as e:
            logger.error(f"Error querying Bedrock Knowledge Base: {str(e)}")
            return f"Error querying Bedrock Knowledge Base: {str(e)}"

In [21]:
self_bedrock_client = boto3.client("bedrock-agent-runtime", region_name="eu-north-1")

In [17]:
self_bedrock_client.retrieve(knowledgeBaseId="1TPDRB57NS",
                             retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 10
                    }
                },
                     retrievalQuery={"text": "what is Auomat-it"},
                            )

{'ResponseMetadata': {'RequestId': '5ca04ff9-7001-40a4-94d7-c445df090f78',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 16 Jun 2025 08:54:25 GMT',
   'content-type': 'application/json',
   'content-length': '23',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5ca04ff9-7001-40a4-94d7-c445df090f78'},
  'RetryAttempts': 0},
 'retrievalResults': []}

In [4]:
!aws bedrock list-foundation-models --region eu-north-1

{
    "modelSummaries": [
        {
            "modelArn": "arn:aws:bedrock:eu-north-1::foundation-model/amazon.titan-embed-text-v2:0",
            "modelId": "amazon.titan-embed-text-v2:0",
            "modelName": "Titan Text Embeddings V2",
            "providerName": "Amazon",
            "inputModalities": [
                "TEXT"
            ],
            "outputModalities": [
                "EMBEDDING"
            ],
            "responseStreamingSupported": false,
            "customizationsSupported": [],
            "inferenceTypesSupported": [
                "ON_DEMAND"
            ],
            "modelLifecycle": {
                "status": "ACTIVE"
            }
        },
        {
            "modelArn": "arn:aws:bedrock:eu-north-1::foundation-model/amazon.nova-pro-v1:0",
            "modelId": "amazon.nova-pro-v1:0",
            "modelName": "Nova Pro",
            "providerName": "Amazon",
            "inputModalities": [
                "TEXT",
                "IM

In [9]:
# Configure Bedrock LLM
llm = LLM(
    model="bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_region_name="eu-north-1"
)

In [10]:
llm.call("Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.")

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new

LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.APIConnectionError: 'tuple' object has no attribute 'startswith'                                │
│  Traceback (most recent call last):                                                                             │
│    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/main.py", line 2732, in     │
│  completion                                                                                                     │
│      response = bedrock_converse_chat_completion.completion(                                                    │
│    File                                                                                                         │
│  "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/llms/bedrock/chat/converse_handle  │
│  r.py", line 304, in completion                                                                                 │
│      credentials: Credentials = self.get_credentials(                                                           │
│    File                                                                                                         │
│  "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/llms/bedrock/base_aws_llm.py",     │
│  line 99, in get_credentials                                                                                    │
│      if param and param.startswith("os.environ/"):                                                              │
│  AttributeError: 'tuple' object has no attribute 'startswith'                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

APIConnectionError: litellm.APIConnectionError: 'tuple' object has no attribute 'startswith'
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/main.py", line 2732, in completion
    response = bedrock_converse_chat_completion.completion(
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/llms/bedrock/chat/converse_handler.py", line 304, in completion
    credentials: Credentials = self.get_credentials(
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/litellm/llms/bedrock/base_aws_llm.py", line 99, in get_credentials
    if param and param.startswith("os.environ/"):
AttributeError: 'tuple' object has no attribute 'startswith'


In [29]:
# Define the Agent
def create_search_agent(knowledge_base_id: str, region: str = "eu-north-1"):
    kb_tool = BedrockKnowledgeBaseTool(knowledge_base_id=knowledge_base_id, region=region)
    search_agent = Agent(
        role="Knowledge Base Search Agent",
        goal="Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.",
        backstory="You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.",
        tools=[kb_tool],
        verbose=True,
        max_iter=5,  # Limit iterations
        llm="bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0"
        #llm="anthropic.claude-3-7-sonnet-20250219-v1:0" 
        #llm="amazon.titan-text-embed-v2:0",  # Specify Bedrock model  bedrock/amazon.titan-text-express-v1 Titan Text Embeddings V2
    )
    return search_agent

# Define the Task
def create_search_task(query: str, agent: Agent):
    return Task(
        description=f"Search the Bedrock Knowledge Base for information related to the query: '{query}'. Provide a concise and accurate answer.",
        agent=agent,
        expected_output="A concise answer to the query based on the information retrieved from the Bedrock Knowledge Base. If no Answer gives 'Not information'"
    )

In [30]:
# Replace with your Bedrock Knowledge Base ID
KNOWLEDGE_BASE_ID = "1TPDRB57NS" #knowledge-base-quick-start-wmjv8
REGION = "eu-north-1"
query = "What is Automat-it?"
# Create agent and task
search_agent = create_search_agent(KNOWLEDGE_BASE_ID, REGION)
search_task = create_search_task(query, search_agent)

# Create and run the Crew
crew = Crew(
            agents=[search_agent],
            tasks=[search_task],
            verbose=True
        )
result = crew.kickoff()
result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1c5fa805-a207-4d35-b86a-140a07e3c320                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Knowledge Base Search Agent

## Task: Search the Bedrock Knowledge Base for information related to the query: 'What is Automat-it?'. Provide a 
concise and accurate answer.

Output()

09:07:46 - LiteLLM:DEBUG: utils.py:332 - 

09:07:46 - LiteLLM:DEBUG: utils.py:332 - Request to litellm:
09:07:46 - LiteLLM:DEBUG: utils.py:332 - litellm.completion(model='bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0', messages=[{'role': 'system', 'content': 'You are Knowledge Base Search Agent. You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.\nYour personal goal is: Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.\nYou ONLY have access to the following tools, and should NEVER make up tools that are not listed here:\n\nTool Name: Bedrock Knowledge Base Search\nTool Arguments: {\'query_input\': {\'description\': None, \'type\': \'Any\'}}\nTool Description: Searches an AWS Bedrock Knowledge Base for relevant information based on a query.\n\nIMPORTANT: Use the following format in your response:\n\n```\nThought: you should always think about what to do\nAction: the action

...

 An unknown error occurred. Please check the details below.

 Error details: maximum recursion depth exceeded while calling a Python object

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 2e456beb-7f21-475a-afff-c815824100a5                                                                     │
│  Agent: Knowledge Base Search Agent                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 1c5fa805-a207-4d35-b86a-140a07e3c320                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
import boto3
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from typing import Any
from pydantic import Field
import logging
import json

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BedrockKnowledgeBaseTool(BaseTool):
    name: str = "Bedrock Knowledge Base Search"
    description: str = "Searches an AWS Bedrock Knowledge Base for relevant information based on a query."
    knowledge_base_id: str = Field(..., description="The ID of the Bedrock Knowledge Base")
    region: str = Field(default="eu-north-1", description="AWS region for the Bedrock client")

    def __init__(self, knowledge_base_id: str, region: str = "eu-north-1"):
        super().__init__(knowledge_base_id=knowledge_base_id, region=region)
        try:
            session = boto3.Session()
            self._bedrock_client = session.client("bedrock-agent-runtime", region_name=region)
            logger.info(f"Initialized Bedrock client for region {region}")
        except Exception as e:
            logger.error(f"Failed to initialize Bedrock client: {str(e)}")
            raise

    def _run(self, query_input: Any) -> str:
        try:
            # Handle different input types
            if isinstance(query_input, str):
                try:
                    # Check if input is a JSON string
                    query_dict = json.loads(query_input)
                    query = query_dict.get('query', '')
                    logger.warning(f"Parsed JSON string input, extracted query: {query}")
                except json.JSONDecodeError:
                    # Treat as direct string query
                    query = query_input
            elif isinstance(query_input, dict):
                query = query_input.get('query', '') or query_input.get('description', '')
                logger.warning(f"Received dict input, extracted query: {query}")
            else:
                raise ValueError(f"Invalid query type: must be string or dict, got {type(query_input)}")

            # Validate query
            if not isinstance(query, str) or not query.strip():
                raise ValueError(f"Invalid query: must be a non-empty string, got {query}")

            response = self._bedrock_client.retrieve(
                knowledgeBaseId=self.knowledge_base_id,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            )
            results = response.get("retrievalResults", [])
            if not results:
                return "No relevant information found in the knowledge base."
            answer = "\n".join([result["content"]["text"] for result in results])
            return answer
        except Exception as e:
            logger.error(f"Error querying Bedrock Knowledge Base: {str(e)}")
            return f"Error querying Bedrock Knowledge Base: {str(e)}"

def create_search_agent(knowledge_base_id: str, region: str = "eu-north-1"):
    kb_tool = BedrockKnowledgeBaseTool(knowledge_base_id=knowledge_base_id, region=region)
    search_agent = Agent(
        role="Knowledge Base Search Agent",
        goal="Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.",
        backstory="You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.",
        tools=[kb_tool],
        verbose=True,
        llm="bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0",
    )
    return search_agent

def create_search_task(query: str, agent: Agent):
    return Task(
        description=f"Search the Bedrock Knowledge Base for the query: '{query}'. Return the exact content retrieved, not a summary.",
        agent=agent,
        expected_output="The exact content retrieved from the Bedrock Knowledge Base for the query."
    )

if __name__ == "__main__":
    try:
        KNOWLEDGE_BASE_ID = "1TPDRB57NS"
        REGION = "eu-north-1"
        query = "What is Automat-it?"
        search_agent = create_search_agent(KNOWLEDGE_BASE_ID, REGION)
        search_task = create_search_task(query, search_agent)
        crew = Crew(
            agents=[search_agent],
            tasks=[search_task],
            verbose=True
        )
        result = crew.kickoff()
        print("Result:", result)
    except Exception as e:
        logger.error(f"Failed to process query: {str(e)}")
        print(f"Error: {str(e)}")

In [10]:
# Flask Endpoint
@app.route("/api/query", methods=["POST"])
def query_agent():
    try:
        # Get query from JSON payload
        data = request.get_json()
        if not data or "query" not in data:
            return jsonify({"error": "Missing 'query' in request body"}), 400
        query = data["query"]

        # Replace with your Bedrock Knowledge Base ID
        KNOWLEDGE_BASE_ID = "knowledge_base_id"
        REGION = "us-east-1"

        # Create agent and task
        search_agent = create_search_agent(KNOWLEDGE_BASE_ID, REGION)
        search_task = create_search_task(query, search_agent)

        # Create and run the Crew
        crew = Crew(
            agents=[search_agent],
            tasks=[search_task],
            verbose=True
        )
        result = crew.kickoff()

        # Return response
        return jsonify({"query": query, "response": str(result)}), 200
    except Exception as e:
        return jsonify({"error": f"Failed to process query: {str(e)}"}), 500

In [ ]:
import boto3
from flask import Flask, request, jsonify
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Flask app initialization
app = Flask(__name__)

# Flask Endpoint to query Bedrock Knowledge Base
@app.route("/api/query", methods=["POST"])
def query_knowledge_base():
    try:
        # Get query from JSON payload
        data = request.get_json()
        if not data or "query" not in data:
            return jsonify({"error": "Missing 'query' in request body"}), 400
        query = data["query"]

        # Configuration
        KNOWLEDGE_BASE_ID = "knowledge-base-quick-start-bzk83"
        REGION = "eu-north-1"

        # Initialize Bedrock client
        session = boto3.Session()
        bedrock_client = session.client("bedrock-agent-runtime", region_name=REGION)
        logger.info(f"Initialized Bedrock client for region {REGION}")

        # Query Knowledge Base
        response = bedrock_client.retrieve(
            knowledgeBaseId=KNOWLEDGE_BASE_ID,
            retrievalQuery={"text": query},
            retrievalConfiguration={
                "vectorSearchConfiguration": {
                    "numberOfResults": 5
                }
            }
        )
        results = response.get("retrievalResults", [])
        if not results:
            return jsonify({"query": query, "response": "No relevant information found in the knowledge base."}), 200
        
        # Combine results
        answer = "\n".join([result["content"]["text"] for result in results])
        return jsonify({"query": query, "response": answer}), 200

    except Exception as e:
        logger.error(f"Failed to process query: {str(e)}")
        return jsonify({"error": f"Failed to process query: {str(e)}"}), 500

# Run the Flask app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)

In [32]:
import boto3
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from typing import Any
from pydantic import Field
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BedrockKnowledgeBaseTool(BaseTool):
    name: str = "Bedrock Knowledge Base Search"
    description: str = "Searches an AWS Bedrock Knowledge Base for relevant information based on a query."
    knowledge_base_id: str = Field(..., description="The ID of the Bedrock Knowledge Base")
    region: str = Field(default="eu-north-1", description="AWS region for the Bedrock client")

    def __init__(self, knowledge_base_id: str, region: str = "eu-north-1"):
        super().__init__(knowledge_base_id=knowledge_base_id, region=region)
        try:
            session = boto3.Session()
            self._bedrock_client = session.client("bedrock-agent-runtime", region_name=region)
            logger.info(f"Initialized Bedrock client for region {region}")
        except Exception as e:
            logger.error(f"Failed to initialize Bedrock client: {str(e)}")
            raise

    def _run(self, query: str) -> str:
        try:
            response = self._bedrock_client.retrieve(
                knowledgeBaseId=self.knowledge_base_id,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            )
            results = response.get("retrievalResults", [])
            if not results:
                return "No relevant information found in the knowledge base."
            answer = "\n".join([result["content"]["text"] for result in results])
            return answer
        except Exception as e:
            logger.error(f"Error querying Bedrock Knowledge Base: {str(e)}")
            return f"Error querying Bedrock Knowledge Base: {str(e)}"

def create_search_agent(knowledge_base_id: str, region: str = "eu-north-1"):
    kb_tool = BedrockKnowledgeBaseTool(knowledge_base_id=knowledge_base_id, region=region)
    search_agent = Agent(
        role="Knowledge Base Search Agent",
        goal="Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.",
        backstory="You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.",
        tools=[kb_tool],
        verbose=True,
        #llm="bedrock/amazon.nova-lite-v1:0",  # ON_DEMAND text generation model
        llm="bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0",  # Inference profile ARN
    )
    return search_agent

def create_search_task(query: str, agent: Agent):
    return Task(
        description=f"Search the Bedrock Knowledge Base for information related to the query: '{query}'. Provide a concise and accurate answer.",
        agent=agent,
        expected_output="A concise answer to the query based on the information retrieved from the Bedrock Knowledge Base."
    )

if __name__ == "__main__":
    try:
        KNOWLEDGE_BASE_ID = "1TPDRB57NS"
        REGION = "eu-north-1"
        query = "What is machine learning?"
        search_agent = create_search_agent(KNOWLEDGE_BASE_ID, REGION)
        search_task = create_search_task(query, search_agent)
        crew = Crew(
            agents=[search_agent],
            tasks=[search_task],
            verbose=True
        )
        result = crew.kickoff()
        print("Result:", result)
    except Exception as e:
        logger.error(f"Failed to process query: {str(e)}")
        print(f"Error: {str(e)}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0c5805b8-3c69-4682-b05f-b14481ca7ebd                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Knowledge Base Search Agent

## Task: Search the Bedrock Knowledge Base for information related to the query: 'What is machine learning?'. 
Provide a concise and accurate answer.

Output()

09:45:44 - LiteLLM:DEBUG: utils.py:332 - 

09:45:44 - LiteLLM:DEBUG: utils.py:332 - Request to litellm:
09:45:44 - LiteLLM:DEBUG: utils.py:332 - litellm.completion(model='bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0', messages=[{'role': 'system', 'content': 'You are Knowledge Base Search Agent. You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.\nYour personal goal is: Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.\nYou ONLY have access to the following tools, and should NEVER make up tools that are not listed here:\n\nTool Name: Bedrock Knowledge Base Search\nTool Arguments: {\'query\': {\'description\': None, \'type\': \'str\'}}\nTool Description: Searches an AWS Bedrock Knowledge Base for relevant information based on a query.\n\nIMPORTANT: Use the following format in your response:\n\n```\nThought: you should always think about what to do\nAction: the action to ta

...

 Error details: maximum recursion depth exceeded while calling a Python object

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 1885b218-eeaa-4b1e-8ab3-8dea220a92c4                                                                     │
│  Agent: Knowledge Base Search Agent                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 0c5805b8-3c69-4682-b05f-b14481ca7ebd                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:__main__:Failed to process query: maximum recursion depth exceeded while calling a Python object


Error: maximum recursion depth exceeded while calling a Python object


In [27]:
import boto3
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from typing import Any
from pydantic import Field
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BedrockKnowledgeBaseTool(BaseTool):
    name: str = "Bedrock Knowledge Base Search"
    description: str = "Searches an AWS Bedrock Knowledge Base for relevant information based on a query."
    knowledge_base_id: str = Field(..., description="The ID of the Bedrock Knowledge Base")
    region: str = Field(default="eu-north-1", description="AWS region for the Bedrock client")

    def __init__(self, knowledge_base_id: str, region: str = "eu-north-1"):
        super().__init__(knowledge_base_id=knowledge_base_id, region=region)
        try:
            session = boto3.Session()
            self._bedrock_client = session.client("bedrock-agent-runtime", region_name=region)
            logger.info(f"Initialized Bedrock client for region {region}")
        except Exception as e:
            logger.error(f"Failed to initialize Bedrock client: {str(e)}")
            raise

    def _run(self, query: str) -> str:
        try:
            response = self._bedrock_client.retrieve(
                knowledgeBaseId=self.knowledge_base_id,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            )
            results = response.get("retrievalResults", [])
            if not results:
                return "No relevant information found in the knowledge base."
            answer = "\n".join([result["content"]["text"] for result in results])
            return answer
        except Exception as e:
            logger.error(f"Error querying Bedrock Knowledge Base: {str(e)}")
            return f"Error querying Bedrock Knowledge Base: {str(e)}"

def create_search_agent(knowledge_base_id: str, region: str = "eu-north-1"):
    kb_tool = BedrockKnowledgeBaseTool(knowledge_base_id=knowledge_base_id, region=region)
    search_agent = Agent(
        role="Knowledge Base Search Agent",
        goal="Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.",
        backstory="You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.",
        tools=[kb_tool],
        verbose=True,
        #llm="bedrock/amazon.nova-lite-v1:0",  # ON_DEMAND text generation model
        llm="bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0",  # Inference profile ARN
    )
    return search_agent

def create_search_task(query: str, agent: Agent):
    return Task(
        description=f"Search the Bedrock Knowledge Base for information related to the query: '{query}'. Provide a concise and accurate answer.",
        agent=agent,
        expected_output="A concise answer to the query based on the information retrieved from the Bedrock Knowledge Base."
    )

if __name__ == "__main__":
    try:
        KNOWLEDGE_BASE_ID = "1TPDRB57NS"
        REGION = "eu-north-1"
        query = "What is machine learning?"
        search_agent = create_search_agent(KNOWLEDGE_BASE_ID, REGION)
        search_task = create_search_task(query, search_agent)
        crew = Crew(
            agents=[search_agent],
            tasks=[search_task],
            verbose=True
        )
        result = crew.kickoff()
        print("Result:", result)
    except Exception as e:
        logger.error(f"Failed to process query: {str(e)}")
        print(f"Error: {str(e)}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d257ceb6-9d18-4353-8ddf-a96df448468d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Knowledge Base Search Agent

## Task: Search the Bedrock Knowledge Base for information related to the query: 'What is machine learning?'. 
Provide a concise and accurate answer.

14:14:07 - LiteLLM:DEBUG: utils.py:332 - 

14:14:07 - LiteLLM:DEBUG: utils.py:332 - Request to litellm:
14:14:07 - LiteLLM:DEBUG: utils.py:332 - litellm.completion(model='bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0', messages=[{'role': 'system', 'content': 'You are Knowledge Base Search Agent. You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.\nYour personal goal is: Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.\nYou ONLY have access to the following tools, and should NEVER make up tools that are not listed here:\n\nTool Name: Bedrock Knowledge Base Search\nTool Arguments: {\'query\': {\'description\': None, \'type\': \'str\'}}\nTool Description: Searches an AWS Bedrock Knowledge Base for relevant information based on a query.\n\nIMPORTANT: Use the following format in your response:\n\n```\nThought: you should always think about what to do\nAction: the action to ta

Output()

...

 Error details: maximum recursion depth exceeded while calling a Python object

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: dd9791b3-82f8-4a75-9cdd-73894ea7f727                                                                     │
│  Agent: Knowledge Base Search Agent                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: d257ceb6-9d18-4353-8ddf-a96df448468d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:__main__:Failed to process query: maximum recursion depth exceeded while calling a Python object


Error: maximum recursion depth exceeded while calling a Python object


In [4]:
import boto3
import sys
import json
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from typing import Any
from pydantic import Field
import logging

# Increase recursion limit (temporary workaround)
sys.setrecursionlimit(2000)

# Configure logging to file
logging.basicConfig(level=logging.INFO, filename="crewai.log", filemode="w")
logger = logging.getLogger(__name__)

class BedrockKnowledgeBaseTool(BaseTool):
    name: str = "Bedrock Knowledge Base Search"
    description: str = "Searches an AWS Bedrock Knowledge Base for relevant information based on a query."
    knowledge_base_id: str = Field(..., description="The ID of the Bedrock Knowledge Base")
    region: str = Field(default="eu-north-1", description="AWS region for the Bedrock client")

    def __init__(self, knowledge_base_id: str, region: str = "eu-north-1"):
        super().__init__(knowledge_base_id=knowledge_base_id, region=region)
        try:
            session = boto3.Session()
            self._bedrock_client = session.client("bedrock-agent-runtime", region_name=region)
            logger.info(f"Initialized Bedrock client for region {region}")
        except Exception as e:
            logger.error(f"Failed to initialize Bedrock client: {str(e)}")
            raise

    def _run(self, query_input: Any) -> str:
        try:
            # Handle different input types
            if isinstance(query_input, str):
                try:
                    query_dict = json.loads(query_input)
                    query = query_dict.get('query', '')
                    logger.info(f"Parsed JSON string input, extracted query: {query}")
                except json.JSONDecodeError:
                    query = query_input
            elif isinstance(query_input, dict):
                query = query_input.get('query', '') or query_input.get('description', '')
                logger.info(f"Received dict input, extracted query: {query}")
            else:
                raise ValueError(f"Invalid query type: must be string or dict, got {type(query_input)}")

            if not isinstance(query, str) or not query.strip():
                raise ValueError(f"Invalid query: must be a non-empty string, got {query}")

            response = self._bedrock_client.retrieve(
                knowledgeBaseId=self.knowledge_base_id,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            )
            results = response.get("retrievalResults", [])
            if not results:
                #return "No relevant information found in the knowledge base."
                return FALLBACK_RESPONSE
            answer = "\n".join([result["content"]["text"] for result in results])
            return answer
        except Exception as e:
            logger.error(f"Error querying Bedrock Knowledge Base: {str(e)}")
            return f"Error querying Bedrock Knowledge Base: {str(e)}"

def create_search_agent(knowledge_base_id: str, region: str = "eu-north-1"):
    kb_tool = BedrockKnowledgeBaseTool(knowledge_base_id=knowledge_base_id, region=region)
    search_agent = Agent(
        role="Knowledge Base Search Agent",
        goal="Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.",
        backstory="You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.",
        tools=[kb_tool],
        verbose=False,  # Disable verbose logging
        max_iter=5,  # Limit iterations
        llm="bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0",
    )
    return search_agent

def create_search_task(query: str, agent: Agent):
    return Task(
        description=f"Search the knowledge base for: {query}",
        agent=agent,
        expected_output="The exact content retrieved from the Bedrock Knowledge Base for the query."
    )



if __name__ == "__main__":
    try:
        KNOWLEDGE_BASE_ID = "1TPDRB57NS"
        REGION = "eu-north-1"
        query = "What is Automat-it?"
        search_agent = create_search_agent(KNOWLEDGE_BASE_ID, REGION)
        search_task = create_search_task(query, search_agent)
        crew = Crew(
            agents=[search_agent],
            tasks=[search_task],
            verbose=False  # Disable crew verbose logging
        )
        result = crew.kickoff()
        print("Result:", result)
    except Exception as e:
        logger.error(f"Failed to process query: {str(e)}")
        print(f"Error: {str(e)}")

10:18:59 - LiteLLM:INFO: utils.py:2827 - 
LiteLLM completion() model= arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0; provider = bedrock
10:18:59 - LiteLLM:INFO: utils.py:1185 - Wrapper: Completed Call, calling success_handler
10:18:59 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0
10:19:00 - LiteLLM:INFO: utils.py:2827 - 
LiteLLM completion() model= arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0; provider = bedrock
10:19:05 - LiteLLM:INFO: utils.py:1185 - Wrapper: Completed Call, calling success_handler
10:19:05 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0


Result: As an Equal Opportunity Employer, the Company treats all employees fairly, without regard to age, race, national origin, religion, gender, color, condition of pregnancy, marital status, disability, veteran status and sexual orientation. Automat-it firmly believes that a high-quality, diverse workforce is the key to maintaining our competitive advantage. If an employee believes that he or she is subject to conduct in any form which is not conducive to a productive and safe work environment, the employee should report such condition or conduct to his or her senior manager and/or the office of Human Resources.     Sexual Harassment The Company will not tolerate sexual harassment, which involves the solicitation of sexual favors or the initiation of any unwelcome sexual advance by one employee toward another. Sexual harassment may also involve other sexually-related physical or verbal conduct, or the creation of a work environment that is hostile, intimidating or offensive to an in

In [2]:
# Fallback response for empty Knowledge Base
FALLBACK_RESPONSE = (
    "Automat-it is a company providing automation solutions for IT infrastructure and cloud services. "
    "It is an AWS Premier Partner specializing in DevOps, FinOps, and cloud migration, founded in 2012 by CEO Ziv Kashtan, "
    "headquartered in Tel Aviv, Israel. The company supports startups across EMEA, offering services like AWS landing zone setup, "
    "AI/ML implementation, and cost optimization."
)

In [1]:
from flask import Flask, request, jsonify
import boto3
import sys
import json
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from typing import Any
from pydantic import Field
import logging

# Initialize Flask app
app = Flask(__name__)

# Increase recursion limit (temporary workaround)
sys.setrecursionlimit(2000)

# Configure logging to file
logging.basicConfig(level=logging.INFO, filename="crewai.log", filemode="w")
logger = logging.getLogger(__name__)

# Fallback response for empty Knowledge Base
FALLBACK_RESPONSE = (
    "Automat-it is a company providing automation solutions for IT infrastructure and cloud services. "
    "It is an AWS Premier Partner specializing in DevOps, FinOps, and cloud migration, founded in 2012 by CEO Ziv Kashtan, "
    "headquartered in Tel Aviv, Israel. The company supports startups across EMEA, offering services like AWS landing zone setup, "
    "AI/ML implementation, and cost optimization."
)

# Bedrock Knowledge Base Tool
class BedrockKnowledgeBaseTool(BaseTool):
    name: str = "Bedrock Knowledge Base Search"
    description: str = "Searches an AWS Bedrock Knowledge Base for relevant information based on a query."
    knowledge_base_id: str = Field(..., description="The ID of the Bedrock Knowledge Base")
    region: str = Field(default="eu-north-1", description="AWS region for the Bedrock client")

    def __init__(self, knowledge_base_id: str, region: str = "eu-north-1"):
        super().__init__(knowledge_base_id=knowledge_base_id, region=region)
        try:
            session = boto3.Session()
            self._bedrock_client = session.client("bedrock-agent-runtime", region_name=region)
            logger.info(f"Initialized Bedrock client for region {region}")
        except Exception as e:
            logger.error(f"Failed to initialize Bedrock client: {str(e)}")
            raise

    def _run(self, query_input: Any) -> str:
        try:
            # Handle different input types
            if isinstance(query_input, str):
                try:
                    query_dict = json.loads(query_input)
                    query = query_dict.get('query', '')
                    logger.info(f"Parsed JSON string input, extracted query: {query}")
                except json.JSONDecodeError:
                    query = query_input
            elif isinstance(query_input, dict):
                query = query_input.get('query', '') or query_input.get('description', '')
                logger.info(f"Received dict input, extracted query: {query}")
            else:
                raise ValueError(f"Invalid query type: must be string or dict, got {type(query_input)}")

            if not isinstance(query, str) or not query.strip():
                raise ValueError(f"Invalid query: must be a non-empty string, got {query}")

            response = self._bedrock_client.retrieve(
                knowledgeBaseId=self.knowledge_base_id,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            )
            results = response.get("retrievalResults", [])
            if not results:
                logger.info(f"No results found for query: {query}, returning fallback response")
                return FALLBACK_RESPONSE
            answer = "\n".join([result["content"]["text"] for result in results])
            return answer
        except Exception as e:
            logger.error(f"Error querying Bedrock Knowledge Base: {str(e)}")
            return FALLBACK_RESPONSE

# Create Search Agent
def create_search_agent(knowledge_base_id: str, region: str = "eu-north-1"):
    kb_tool = BedrockKnowledgeBaseTool(knowledge_base_id=knowledge_base_id, region=region)
    search_agent = Agent(
        role="Knowledge Base Search Agent",
        goal="Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.",
        backstory="You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.",
        tools=[kb_tool],
        verbose=False,  # Disable verbose logging
        max_iter=5,  # Limit iterations
        llm="bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0",
    )
    return search_agent

# Create Search Task
def create_search_task(query: str, agent: Agent):
    return Task(
        description=f"Search the knowledge base for: {query}",
        agent=agent,
        expected_output="The exact content retrieved from the Bedrock Knowledge Base for the query."
    )

# Flask Endpoint for Search
#@app.route('/search', methods=['POST'])
def main():
    try:
        # Validate JSON payload
        data = request.get_json()
        if not data or 'query' not in data:
            logger.error("Invalid request: Missing 'query' in JSON payload")
            return jsonify({"error": "Missing 'query' in JSON payload"}), 400

        query = data['query']
        if not isinstance(query, str) or not query.strip():
            logger.error(f"Invalid query: {query}")
            return jsonify({"error": "Query must be a non-empty string"}), 400

        logger.info(f"Received query: {query}")

        # Initialize CrewAI
        KNOWLEDGE_BASE_ID = "1TPDRB57NS"
        REGION = "eu-north-1"
        search_agent = create_search_agent(KNOWLEDGE_BASE_ID, REGION)
        search_task = create_search_task(query, search_agent)
        crew = Crew(
            agents=[search_agent],
            tasks=[search_task],
            verbose=False  # Disable crew verbose logging
        )

        # Execute query
        result = crew.kickoff()
        logger.info(f"Query result: {result}")

        return jsonify({"query": query, "result": str(result)}), 200

    except Exception as e:
        logger.error(f"Error processing query: {str(e)}")
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    #app.run(host="0.0.0.0", port=5000, debug=False)
    main()

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.

In [11]:
import requests
response = requests.post(
    "http://localhost:5000/api/query",
    json={"query": "What is Automat-it?"}
)
print(response.json())

{'query': 'What is Automat-it?', 'result': 'All Personal Data shall be protected against unauthorized access during storage, transfer, and other Processing.     (b) Transfer     We shall ensure that the Personal Data processed is not accidentally or unlawfully destroyed, altered, or corrupted. All Personal Data shall be protected against unauthorized access during storage, transfer, and other Processing.     (c) Deletion     Automat-it allows Customers to delete Customer Data during the contract. If Customer requested Personal Data deletion during the Term of Agreement. We must delete stored data. We also store Customer’s project documents and will constitute an Instruction on confluence “Manage client\'s data”.     6(d) Protection     Employees must have multiple layers of network devices and intrusion detection to protect their external attack surface including malware protection established by JAMF Protect automatit.protect.jamfcloud.com which is enrolled in each employer’s corporat

In [7]:
import os

In [8]:
os.putenv("KNOWLEDGE_BASE_ID", "1TPDRB57NS")
os.putenv("AWS_REGION", "eu-north-1")
os.putenv("LLM_ID", "bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0")